<a href="https://colab.research.google.com/github/talkin24/nlp_with_BERT_GPT/blob/main/examples/pair_classification/deploy_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지 설치
pip 명령어로 의존성 있는 패키지를 설치합니다.



In [1]:
!pip install ratsnlp

     |████████████████████████████████| 42 kB 918 kB/s 
     |████████████████████████████████| 57 kB 3.2 MB/s 
     |████████████████████████████████| 806 kB 45.6 MB/s 
     |████████████████████████████████| 2.8 MB 31.7 MB/s 
     |████████████████████████████████| 398 kB 72.8 MB/s 
     |████████████████████████████████| 134 kB 67.2 MB/s 
     |████████████████████████████████| 829 kB 62.6 MB/s 
     |████████████████████████████████| 636 kB 40.1 MB/s 
     |████████████████████████████████| 895 kB 40.1 MB/s 
     |████████████████████████████████| 3.3 MB 46.4 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 1.1 MB 40.5 MB/s 
     |████████████████████████████████| 96 kB 5.0 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 271 kB 64.3 MB/s 
     |████████████████████████████████| 144 kB 55.6 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sh

# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [7]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments
args = ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-paircls",
    max_seq_length=64,
)

Exception: ignored

# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

In [ ]:
import torch
from transformers import BertConfig, BertForSequenceClassification
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)
model = BertForSequenceClassification(pretrained_model_config)
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

# 인퍼런스 함수 선언
인퍼런스 함수를 선언합니다.

In [ ]:
def inference_fn(premise, hypothesis):
    inputs = tokenizer(
        [(premise, hypothesis)],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        prob = outputs.logits.softmax(dim=1)
        entailment_prob = round(prob[0][0].item(), 2)
        contradiction_prob = round(prob[0][1].item(), 2)
        neutral_prob = round(prob[0][2].item(), 2)
        if torch.argmax(prob) == 0:
            pred = "참 (entailment)"
        elif torch.argmax(prob) == 1:
            pred = "거짓 (contradiction)"
        else:
            pred = "중립 (neutral)"
    return {
        'premise': premise,
        'hypothesis': hypothesis,
        'prediction': pred,
        'entailment_data': f"참 {entailment_prob}",
        'contradiction_data': f"거짓 {contradiction_prob}",
        'neutral_data': f"중립 {neutral_prob}",
        'entailment_width': f"{entailment_prob * 100}%",
        'contradiction_width': f"{contradiction_prob * 100}%",
        'neutral_width': f"{neutral_prob * 100}%",
    }

# 웹서비스 만들기 준비

`ngrok`은 코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구입니다. `ngrok`을 실행하려면 [회원가입](https://dashboard.ngrok.com/signup) 후 [로그인](https://dashboard.ngrok.com/login)을 한 뒤 [이곳](https://dashboard.ngrok.com/get-started/your-authtoken)에 접속해 인증 토큰(authtoken)을 확인해야 합니다. 예를 들어 확인된 `authtoken`이 `test111`이라면 다음과 같이 실행합니다.

```bash
!mkdir /root/.ngrok2 && echo "authtoken: test111" > /root/.ngrok2/ngrok.yml
```

In [ ]:
!mkdir /root/.ngrok2 && echo "authtoken: {이곳에 확인된 인증 토큰을 입력하세요}" > /root/.ngrok2/ngrok.yml

# 웹서비스 개시
아래처럼 실행해 인퍼런스 함수를 웹서비스로 만듭니다.

In [ ]:
from ratsnlp.nlpbook.paircls import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()